## Amazon Review Data : Data Exploration

### Setup spark

In [1]:
import os, pickle, glob
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext

In [2]:
sc = SparkSession.builder \
    .config("spark.driver.memory", "16g") \
	.config("spark.executor.memory", "32g") \
    .config('spark.executor.instances', 4) \
	.appName("Amazon Reviews") \
	.getOrCreate()

In [3]:
sqlContext = SQLContext(sc)

/usr/local/spark/python/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


## Read Data

### Get files

In [4]:
#######################################################################################
###################### change path to: "../clin6/amazon_data" #########################
#######################################################################################
path = "amazon_data"

In [5]:
def read(path):
    """
    Method that loads data file as df
    Takes in 1 parameter: path
    """
    return sc.read.csv(path, sep = "\t", header = True, inferSchema = True)

def get_path(file):
    """
    Method to create path
    Takes in 1 parameter: file name
    """
    return "amazon_data/%s" % file

In [6]:
dir = os.listdir(path)
files = [f for f in dir if os.path.isfile(os.path.join(path, f))]

In [7]:
def get_df(files):
    """
    Method that combines files into 1 big df
    Takes in 1 parameter: list of file names
    """
    df = read(get_path(files[0]))
    n = len(files)
    for i in range(1, n):
        data = read(get_path(files[i]))
        df = df.union(data)
    return df

### Get df and Removing Repetitive/Unnecessary Information

In [8]:
df = get_df(files).drop('marketplace', 'vine')
sqlContext.registerDataFrameAsTable(df, "df")

# Data Exploration Start
* Perform the data exploration step (i.e. evaluate your data, # of observations, details about your data distributions, scales, missing data, column descriptions) Note: For image data you can still describe your data by the number of classes, # of images, plot example classes of the image, size of images, are sizes uniform? Do they need to be cropped? normalized? etc.

In [9]:
### Show how our data looks like
df.show(5)

+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+-----------------+--------------------+--------------------+-----------+
|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|verified_purchase|     review_headline|         review_body|review_date|
+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+-----------------+--------------------+--------------------+-----------+
|   36075342| RAB23OVFNCXZQ|B00LPRXQ4Y|     339193102|17" 2003-2006 For...|      Automotive|          1|            0|          0|                Y|     As it was used,|As it was used, t...| 2015-08-31|
|   42462164|R3NORADVJO6IE6|B000C7S0TO|     907684644|Spectra Premium C...|      Automotive|          5|            0|          0|                Y|          Five Stars|Put it in fine, n..

In [10]:
### Total number of observations
print('There is a total of %d observations' % df.count())

There is a total of 109830520 observations


In [11]:
### Get columns
columns = df.columns
print('Columns:')
print(df.columns)

Columns:
['customer_id', 'review_id', 'product_id', 'product_parent', 'product_title', 'product_category', 'star_rating', 'helpful_votes', 'total_votes', 'verified_purchase', 'review_headline', 'review_body', 'review_date']


In [12]:
### Get number of columns
num_cols = len(columns)
print('There is a total of %d columns' % num_cols)

There is a total of 13 columns


In [13]:
### Get number of missing values for each column
for i in range(num_cols):
    missing = df.filter(df[columns[i]].isNull()).count()
    print("'%s' column has %d missing values" % (columns[i], missing))

'customer_id' column has 0 missing values
'review_id' column has 0 missing values
'product_id' column has 0 missing values
'product_parent' column has 0 missing values
'product_title' column has 0 missing values
'product_category' column has 1753 missing values
'star_rating' column has 1787 missing values
'helpful_votes' column has 1794 missing values
'total_votes' column has 1794 missing values
'verified_purchase' column has 1794 missing values
'review_headline' column has 2044 missing values
'review_body' column has 12438 missing values
'review_date' column has 8243 missing values


In [14]:
data_types = df.dtypes
for i in range(num_cols):
    print("'%s' column is of type '%s'" % (data_types[i][0], data_types[i][1]))

'customer_id' column is of type 'int'
'review_id' column is of type 'string'
'product_id' column is of type 'string'
'product_parent' column is of type 'int'
'product_title' column is of type 'string'
'product_category' column is of type 'string'
'star_rating' column is of type 'string'
'helpful_votes' column is of type 'int'
'total_votes' column is of type 'int'
'verified_purchase' column is of type 'string'
'review_headline' column is of type 'string'
'review_body' column is of type 'string'
'review_date' column is of type 'date'


In [15]:
### Show data distributions
df.describe().show()

+-------+--------------------+--------------+--------------------+--------------------+--------------------+--------------------+-----------------+------------------+-----------------+-----------------+--------------------+--------------------+
|summary|         customer_id|     review_id|          product_id|      product_parent|       product_title|    product_category|      star_rating|     helpful_votes|      total_votes|verified_purchase|     review_headline|         review_body|
+-------+--------------------+--------------+--------------------+--------------------+--------------------+--------------------+-----------------+------------------+-----------------+-----------------+--------------------+--------------------+
|  count|           109830520|     109830520|           109830520|           109830520|           109830520|           109828767|        109828733|         109828726|        109828726|        109828726|           109828476|           109818082|
|   mean| 2.83900989

In [16]:
df = df.select([col for col in df.columns if col not in ['marketplace', 'vine']]).cache()

### var after updated df
columns = df.columns.broadcast((
num_cols = len(columns)

SyntaxError: '(' was never closed (415290016.py, line 4)

In [ ]:
### Get number of missing values for each column
for i in range(num_cols):
    missing = df.filter(df[columns[i]].isNull()).count()
    print("'%s' column has %d missing values" % (columns[i], missing))

In [ ]:
data_types = df.dtypes
for i in range(num_cols):
    print("'%s' column is of type '%s'" % (data_types[i][0], data_types[i][1]))

In [ ]:
### Show data distributions
df.describe().show()

In [ ]:
### Check for duplicates
df.groupBy(columns).count().where('count > 1').show()

In [17]:
### Check for duplicates
#duplicate_rows = df.groupBy(columns).count().where('count > 1')
#duplicate_rows.show()

# Plot
reference:
https://plotly.com/python/v3/apache-spark/

In [18]:
### Import libraries to plot
import plotly.offline as py
import plotly.graph_objs as go
from plotly.offline import iplot
py.init_notebook_mode(connected = True)

from pyspark.sql.functions import col

# Methods to plot

In [19]:
def plot(title, data):
    """
    Helper method to plot
    Takes in two paramaters: title, data
    """
    layout = go.Layout(title = title)                       # Create layout
    fig = go.Figure(data = data, layout = layout)           # Create figure
    iplot(fig)                                              # Plot figure

In [20]:
def plot_histogram(df, i):
    """
    Method to plot histogram of column
    Takes in two parameters: df, column index
    """
    column_name = columns[i]                                # Get column name and data
    data = [go.Histogram(x = df.select(col(column_name)).rdd.flatMap(lambda x: x).collect())]
    title = 'Histogram of %s' % columns[i]                  # Set title
    plot(title, data)                                       # Use helper method to plot

In [21]:
def plot_piechart(df, i):
    """
    Method to plot pie chart of column
    Takes in two parameters: df, column index
    """
    column_name = columns[i]                                                      # Get column name
    counts = df.select(col(column_name)).groupBy(column_name).count().toPandas()  # Get counts
    data = [go.Pie(labels = counts[column_name], values = counts['count'])]       # Get data
    title = 'Pie Chart of %s' % columns[i]                                        # Set title
    plot(title, data)                                       # Use helper method to plot

In [22]:
def plot_barplot(df, i):
    """
    Method to plot bar plot of column
    Takes in one parameter: column index
    """
    column_name = columns[i]                                                      # Get column name 
    counts = df.select(col(column_name)).groupBy(column_name).count().toPandas()  # Get counts
    data = [go.Bar(x = counts[column_name], y = counts['count'])]                 # Get data    
    title = 'Bar Plot of %s' % columns[i]                                         # Set title
    plot(title, data)                                       # Use helper method to plot

In [23]:
def plot_boxplot(df, i):
    """
    Method to plot box plot of column
    Takes in two parameters: df, column index
    """
    column_name = columns[i]                                # Get column name and data
    data = [go.Box(y = df.select(column_name).sample(False, 0.1).rdd.flatMap(lambda x: x).collect())]
    title = 'Box plot of %s' % column_name                  # Set title
    plot(title, data)                                       # Use helper method to plot

In [24]:
def plot_boxplot_no_outliers(df, i):
    """
    Method to plot box plot of column after removing outliers
    Takes in two parameters: df, column index
    """
    column_name = columns[i]                                             # Get column name
    quartiles = df.approxQuantile(column_name, [0.25, 0.75], 0.01)       # Calculate quartiles
    q1 = quartiles[0]
    q3 = quartiles[1]
    iqr = q3 - q1                                                        # Calculate iqr
    lower_bound = q1 - 1.5 * iqr                                         # Define bounds
    upper_bound = q3 + 1.5 * iqr
    
    # Remove outliers
    column_data_no_outliers = df.filter((col(column_name) >= lower_bound) & (col(column_name) <= upper_bound))
    
    # Filter data
    data = [go.Box(y = column_data_no_outliers.select(column_name).sample(False, 0.1).rdd.flatMap(lambda x: x).collect())]
    
    title = 'Box plot of %s (Outliers Removed)' % columns[i]             # Set title
    plot(title, data)                                                    # Use helper method to plot

## Analysis

In [ ]:
### CHANGE COLUMN INDEX TO PLOT
i = 7
plot_histogram(df, i)

* There are clearly noticeable high outliers, which conceal the visibility of the distribution
* Lets look at a boxplot

In [ ]:
### CHANGE COLUMN INDEX TO PLOT
i = 7
plot_boxplot(df, i)

* Indeed we see many high outliers

In [ ]:
### CHANGE COLUMN INDEX TO PLOT
i = 7
plot_boxplot_no_outliers(df, i)

* Outliers removed box plot, we see most reviews have no votes

In [ ]:
### CHANGE COLUMN INDEX TO PLOT
i = 6
plot_barplot(df, i)

* Majority of ratings given are 5 stars
* Second most given rating is 1 but still much less than 5
* Increased ratings from 2 to 5 stars

# Look at category proportions

In [ ]:
### CHANGE COLUMN INDEX TO PLOT
i = 6
plot_piechart(df, i)

# Compare purchase counts by year

In [ ]:
from pyspark.sql.functions import year, substring
import matplotlib.pyplot as plt


def plot_purchase_counts(df, df_name, target_year):
    """
    Plot the purchase counts for a given DataFrame and year.

    Args:
    - df: The DataFrame containing the review data.
    - target_year: The year for which purchase counts will be plotted.
    - df_name: The name of the DataFrame.

    Returns:
    - None
    """
    # Filter the DataFrame to include only rows from the specified year
    df_year = df.filter(year(df['review_date']) == target_year)

    # Extract the month from the 'review_date' column
    df_year = df_year.withColumn('review_month', substring(df_year['review_date'], 6, 2))

    # Count the number of purchases for each month
    purchase_counts = df_year.groupby('review_month').count().orderBy('review_month')

    # Plot the counts using a bar plot
    plt.figure(figsize=(10, 6))
    plt.bar(purchase_counts.toPandas()['review_month'], purchase_counts.toPandas()['count'])
    plt.title(f'Purchase Counts for {df_name} Year {target_year}')
    plt.xlabel('Month')
    plt.ylabel('Number of Purchases')
    plt.show()

In [ ]:
plot_purchase_counts(df, "df", 2015)
plot_purchase_counts(df, "df", 1999)